# Simple model solution: 0.0305221- Top 5%
from https://www.kaggle.com/c/santander-product-recommendation/forums/t/26789/simple-model-solution-0-0305221-top-5


Hi Kagglers !

First, thanks to everybody and all the help that was shared on the forum.

I wanted to expose my solution as it is a single xgboost model which got me quite far. I think, by its simplicity, that it can be interesting to some.

Overview : Single XGBoost model trained on May to December, lag5& lag3 of all products, lag-1 of consumer characteristics from the test file and... that's pretty much it

1. Cleaning : Stripping of all columns, split of dates, removing consumer with missing rows and once the featuring is done, those who will buy nothing. As XGBoost handles NA, i did not go any farther

2. Features : missing renta filled with average of the province (Thanks Alan (AJ) Pryor, Jr. !), 'will_buy' column with the bought products of next month, total products, Lag5 & 3 of all products (did not have time to add all the lags, unfortunately), lag2 of age, renta, total_products and future characteristics from next month.

All categoricals where One Hot encoded, and their values with the less impact (canal_entrada & nomprov) agregated in an "other" column instead of being ignored

-- Cleaning and feature generation were processed in chunks to allow fast iteration / testing

3. Fit : With may to December, 5 folds CV, mlogloss metric

So, pretty simple and in the end, quite efficient

Special thanks to Alan (AJ) Pryor, Jr. !, Tom Van de Wiele and BreakfastPirate for their huge help on the forum

*Additional info:*

Lag for products was a binary indicator saying "Did I have this product in the last n months". For total_products, renta and age it was the minimum of the last n months. 

The "will_buy" was what I wanted to predict, it was the positive difference of current and futur owned products, concatenated as bytes, splitted later if more than one product was added (to only have one product per line, then encoded to be fed to the model

I dropped the users with missing months. I did not have time to see this it was relevant or not, but as my lagged feature were done with a rolling max or min it seemed more correct like that.



In [ ]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [47]:
import os
import numpy as np
import pandas as pd

import logging
logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO)

import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.path.append("../common")

from utils import to_yearmonth, _sub_months_ym, _add_months_ym, TARGET_LABELS, dummies_to_str
from utils import target_str_to_labels, decimal_to_dummies, targets_str_to_indices, targets_dec_to_indices
from utils import load_data2, minimal_clean_data_inplace, preprocess_data_inplace

In [37]:
from utils import FEATURES_NAMES
from dataset import TRAIN_FILE_PATH, TEST_FILE_PATH

In [41]:
training_months = [
    201505,
    201506,
    201507,
    201508,
    201509,
    201510,
    201511,
    201512,
]

user_lag_indices = [2, 3, 5]

# used_training_months = []
# for ym in training_months:
    
#     for lag in user_lag_indices:
#         ym_lag = _sub_months_ym(ym, lag)
#         used_training_months.append(ym_lag)
#     used_training_months.append(ym)
    
# used_training_months = sorted(list(set(used_training_months)))
# print used_training_months

In [40]:
NEXT_FEATURES_NAMES = ['next_' + s for s in FEATURES_NAMES]
def LAG_TARGETS(i):
    return ['lag_%i_' + s for s in TARGET_LABELS]

In [42]:
# list of N_MAX_USERS * len(training_months) lines
# Each line contains : 
# - 'ncodpers', 
# - 'yearmonth', 
# - user features (22), 
# - 'will_buy' column with the bought products of next month (24)
# - total products, 
# - Lag 5 of all products (24), 
# - Lag 3 of all products (24), 
# - lag2 of age, renta, total_products and 
# - future characteristics from next month.

columns = ['ncodpers', 'yearmonth'] +\
        FEATURES_NAMES +\
        ['will_buy', 'total_products'] +\
        LAG_TARGETS(3) + LAG_TARGETS(5) +\
        ['lag_2_age', 'lag_2_renta', 'lag_2_total_products'] +\
        NEXT_FEATURES_NAMES

final_df = pd.DataFrame(columns=columns)

In [43]:
df = None
for ym in training_months:
    ym_list = []
    for lag in user_lag_indices:
        ym_lag = _sub_months_ym(ym, lag)
        ym_list.append(ym_lag)
    ym_list.append(ym)
    ym_list.append(_add_months_ym(ym))
    
    df = load_data2(TRAIN_FILE_PATH, ym_list)
    minimal_clean_data_inplace(df)
    preprocess_data_inplace(df)
    
    df.loc[:, 'fecha_dato'] = df['fecha_dato'].apply(to_yearmonth)
    df.loc[:, 'total_products'] = df[TARGET_LABELS].sum(axis=1)
    
    df = df.sort_values(['ncodpers', 'fecha_dato'])
    
    break

INFO:root:- Number of lines with unknown data : 14628
INFO:root:- Number of columns with nan : 10


In [53]:
df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,total_products
839976,201502,15889,1,0,0,56.0,1995-01-16,0,241,1.0,...,0,0,0,0,1,0,0.0,0.0,0,3.0
421110,201503,15889,1,0,0,56.0,1995-01-16,0,242,1.0,...,0,0,0,0,1,0,0.0,0.0,0,3.0
1888558,201505,15889,1,0,0,56.0,1995-01-16,0,244,1.0,...,0,0,0,1,1,0,0.0,0.0,0,4.0
839975,201502,15890,2,0,0,62.0,1995-01-16,0,241,1.0,...,1,0,0,1,0,0,1.0,1.0,1,8.0
421109,201503,15890,2,0,0,62.0,1995-01-16,0,242,1.0,...,1,0,0,1,0,0,1.0,1.0,1,8.0


In [52]:
df.loc[:, 'total_products'] = df[TARGET_LABELS].sum(axis=1)

In [23]:
from raw_dataset import parse_line, get_user, get_yearmonth, get_target_labels
from raw_dataset import clean_data_inplace, process_data_inplace

In [26]:
def process_row(input_row):
    row = list(input_row)
    if clean_data_inplace(row):
        # process_data_inplace(row)
        return row
    return None

In [22]:
# Load training data
N_MAX_USERS = 10
USERS = {}

# list of N_MAX_USERS * len(training_months) lines
# Each line contains : 
# 0 - 'ncodpers', 
# 1 - 'yearmonth', 
# 2-24 - user features (22), 
# 24 - 'will_buy' column with the bought products of next month, 
# 25 - total products, 
# 26-50 - Lag 5 of all products (24), 
# 50-74 - Lag 3 of all products (24), 
# 74, 75, 76 - lag2 of age, renta, total_products and 
# 77-98 - future characteristics from next month.
DATA = np.zeros((N_MAX_USERS  * len(training_months), 98))

reader = open(TRAIN_FILE_PATH, 'r')
target_labels = get_target_labels(reader.readline())

total = 0
line_counter = 0
removed_rows = 0
while True:
    line = reader.readline()[:-1]
    total += 1

    if line == '':
        break
    row = parse_line(line)
    ym = get_yearmonth(row)
    
    if ym not in used_training_months:
        if ym > training_months[-1]:
            break
        continue
    
    processed_row = process_row(row)
    if processed_row is None:
        removed_rows += 1
        continue

    user_id = get_user(process_row)
    if len(USERS) > N_MAX_USERS and user_id not in USERS:
        continue
  
    user_profile = get_profile(processed_row)
    user_choices = get_choices(processed_row)
    
    if user_id not in USERS:
        USERS[user_id] = line_counter
        DATA[line_counter, 0] = user_id

    if 
    # Set lags :
    
        
    if DATA[line_counter, 0] == user_id
        DATA
    
    
    
#     USERS[user_id] = line_counter
#     DATA
    
        
    if total % 100000 == 0:
        logging.info('-- Processed {} lines : Current month : {}'.format(total, yearmonth_str))

    if total > 10:
        break
    
reader.close()
    
    



[201412, 201501, 201502, 201503, 201504, 201505, 201506, 201507, 201508, 201509, 201510, 201511, 201512]
